Standortkarten Print Skript

In [6]:
import sys
import os
import psycopg2
import time
import configparser
import requests
import json
import subprocess
from requests.auth import HTTPBasicAuth
import urllib2
from ipywidgets import FloatProgress
from IPython.display import display
import csv

In [7]:
def process_exists(process_name):
    call = 'TASKLIST', '/FI', 'imagename eq %s' % process_name
    output = subprocess.check_output(call)
    last_line = output.strip().split('\r\n')[-1]
    return last_line.lower().startswith(process_name.lower())

DatenbankServerFunktionen
@todo: Auslagern in eine Klasse

In [8]:
def openDataBaseServer(config):
    if not process_exists("postgres.exe"):
        print "Datenbank ist nicht offen!"
        processcall=config.get("postgres","bin_dir")+"/bin/postgres.exe "
        processcall+="-D "+config.get("postgres","db_dir")+" "
        processcall+="-p "+config.get("postgres","port")
        
        subprocess.Popen(processcall,shell=True)
        time.sleep(10)
        #@todo check if really open!
        print("Datenbank offen!")
def killDataBaseServer():
    p1=subprocess.Popen("TASKKILL /IM postgres.exe > NUL 2>&1",shell=True)
    p1.wait()
    print "Datenbankserver beendet"
def initDataBaseServer(config):
    if os.path.isdir(config.get("postgres","db_dir"))==False:
        print "Datenbank ist nicht eingerichtet, richte ein!"
        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/initdb --locale=German_Germany.1252 --encoding=UTF8 "
        processcall+=config.get("postgres","db_dir")
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()
        datenbank=openDataBaseServer(config)
        processcall=config.get("postgres","bin_dir")+"/bin/createuser.exe -s -d -w "+config.get("postgres","user")
        p1=subprocess.Popen(processcall,shell=True)
        p1.wait()
        print "Nutzer eingerichtet"

In [9]:
def download_file(url,dest):
    urllib2.install_opener(
            urllib2.build_opener(
                urllib2.ProxyHandler({'https': config.get("general","proxy_https")})
            )
        )
        
    file_name = "tempdata/"+osm_url.split('/')[-1]
    u = urllib2.urlopen(osm_url)
    f = open(file_name, 'wb')
    meta = u.info()
    file_size = int(meta.getheaders("Content-Length")[0])
    print "Downloading: %s Bytes: %s" % (file_name, file_size)

    file_size_dl = 0
    block_sz = 8192

    progressbar = FloatProgress(min=0, max=100) # instantiate the bar
    display(progressbar) # display the bar

    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break
        file_size_dl += len(buffer)
        f.write(buffer)
        progressbar.value=file_size_dl * 100. / file_size
    f.close()

In [10]:
def checkDataBase(config,osm_url,databasename):
    initDataBaseServer(config)
    openDataBaseServer(config)

    #beispielhaft für Baden_Württemberg
    #DB ist OSM_BW_1
    try:
        conn = psycopg2.connect("dbname='"+databasename+"' user='"
                                +config.get("postgres","user")
                                +"' host='"+config.get("postgres","host")
                                +"' password='"+config.get("postgres","password")+"'")
        print "Datenbank existiert"
        #todo: hier noch checken, ob entsprechende Tabellen wirklich existieren
        conn.close()
    except:
        print "Datenbank existiert nicht!"
        #download osm data if not existent
        if not os.path.exists("tempdata"):
            os.makedirs("tempdata")
        if not os.path.isfile("tempdata/"+os.path.basename(osm_url)):
            print "downloading OSM file"
            download_file(osm_url,"tempdata/"+os.path.basename(osm_url))
            print "Download abgeschlossen"
        #create database
        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/psql -w -U "+config.get("postgres","user")
        processcall+=" -p "+config.get("postgres","port")
        processcall+=" -d postgres -c \"CREATE DATABASE "+databasename+"\""

        print processcall
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()

        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/psql -w -U "+config.get("postgres","user")
        processcall+=" -d \""+databasename+"\""
        processcall+=" -c \"CREATE EXTENSION postgis;\""
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()

        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/psql -w -U "+config.get("postgres","user")
        processcall+=" -d \""+databasename+"\""
        processcall+=" -c \"CREATE EXTENSION postgis_sfcgal;\""
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()

        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/psql -w -U "+config.get("postgres","user")
        processcall+=" -d \""+databasename+"\""
        processcall+=" -c \"CREATE EXTENSION postgis_topology;\""
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()

        processcall=config.get("postgres","bin_dir")
        processcall+="/bin/psql -w -U "+config.get("postgres","user")
        processcall+=" -d \""+databasename+"\""
        processcall+=" -c \"CREATE EXTENSION hstore;\""
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()

        #import data
        processcall=config.get("osm2pgsql","bin_path")
        processcall+=" -d "+databasename
        processcall+=" -S "+config.get("osm2pgsql","schema")
        processcall+=" -k --hstore-match-only -r pbf -s"
        processcall+=" -H "+config.get("postgres","host")
        processcall+=" -P "+config.get("postgres","port")
        processcall+=" -U "+config.get("postgres","user")
        processcall+=" -C "+config.get("osm2pgsql","cachesize")
        processcall+=" -c  \""+os.path.abspath("tempdata/"+os.path.basename(osm_url))+"\""
        print processcall
        print "importiere OSM Daten in Datenbank, die kann lange Zeit dauern. Bitte Fenster offenlassen!"
        p1 = subprocess.Popen(processcall,shell=True)
        p1.wait()
        print "Import beendet"

In [11]:
def renderBounds(lon,lat,lon2,lat2,name):
    bbox = mapnik.Box2d(lon, lat, lon2, lat2)
    m.zoom_to_box(bbox)
    print "This is the Bounding Box: ",m.envelope()," and the scale: ",m.scale()
    mapnik.render_to_file(m,name+".png","png",kartenscale)

def printBounds(lon,lat,lon2,lat2):
    print str(lon)+","+str(lat)+","+str(lon2)+","+str(lat2)

In [12]:
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = stdout
#disable proxy for localhost
os.environ['NO_PROXY'] = '127.0.0.1'
config=configparser.ConfigParser()

In [42]:
try:
    config.read("printconfig.ini")
except:
    print "exception while reading config file"

hole Standorte

In [18]:
def holeStandorte(config):
    csvfile='tempdata/standorte.csv'
    textfile='tempdata/text_v2_1_1.txt'
    
    werke={"HW":"Hauptwerk",
           "ZW":"Zweigwerk",
           "HZW":"Haupt-/Zweigwerk",
           "GH":"Gebietshauptwerk"
          }

    f = open(csvfile,"wb")
    st_text_file=open(textfile,"wb")
    st_text=""

    csv_file = csv.writer(f, quotechar='"', quoting=csv.QUOTE_ALL)
    r=requests.post(config.get("webservice","url"),
                    auth=HTTPBasicAuth(config.get("webservice","username"),config.get("webservice","password")),
                    data={"command":"fetchStandortData","suffix":"_v2","branche":"1","bundesland":1},
                   )
    if r.status_code == 200:
        data=json.loads(r.text)
        count=0
        for standort in data['standorte']:
            if count==0:
                csv_file.writerow(standort.keys())
            csv_file.writerow(standort.values())
            st_text+="#"+str(standort['id'])+" "
            st_text+="("+werke[standort['Art']]+")\n"
            st_text+=standort['Name1']+" "+standort['Name2']+" "+standort['Name3']+"\n"
            st_text+=standort['Strasse']+"\n"+standort['PLZStrasse']+" "+standort['Ort']+"\n"
            st_text+=standort['Telefon']+"\n"+standort['Email']+"\n"+standort['Internet']+"\n\n"
            count+=1
    else:
        print "konnte Standorte nicht holen",r.status_code,r.text
    st_text_file.write(st_text)
    f.close()
    st_text_file.close()
    print "Standort CSV und Text geschrieben"
    return os.path.abspath(csvfile),os.path.abspath(textfile)

In [51]:
databasename=config.get("1","db_name")
osm_url=config.get("1","osm_url")
bounds=config.get("1","bounds");
iconGH=config.get("Transportbeton","iconGH")
iconHZW=config.get("Transportbeton","iconHZW")
iconZW=config.get("Transportbeton","iconZW")
iconHW=config.get("Transportbeton","iconHW")
standorte,textfile=holeStandorte(config)


if not os.path.exists("tempdata"):
    os.makedirs("tempdata")
with open('layerdata/OSM_template.xml', 'r') as template_file:
  styledata = template_file.read()
styledata=styledata.replace("$DATABASE",databasename)
styledata=styledata.replace("$BOUNDSSHAPE",os.path.abspath("layerdata/"+bounds))
styledata=styledata.replace("$PATH",os.path.abspath(os.path.dirname("tempdata/../")))
styledata=styledata.replace("$ICON_GH",os.path.abspath("layerdata/"+iconGH))
styledata=styledata.replace("$ICON_HZW",os.path.abspath("layerdata/"+iconHZW))
styledata=styledata.replace("$ICON_ZW",os.path.abspath("layerdata/"+iconZW))
styledata=styledata.replace("$ICON_HW",os.path.abspath("layerdata/"+iconHW))
styledata=styledata.replace("$FILE_STANDORTE",os.path.abspath(standorte))

outfile = open("tempdata/stylesheet.xml", "w")
outfile.write(styledata)
outfile.close()

print "Stylefile written"  

Standort CSV und Text geschrieben
Stylefile written


In [ ]:
checkDataBase(config,osm_url,databasename)
openDataBaseServer(config)

path_temp_files="tempdata/renderedFiles"

if not os.path.exists(path_temp_files):
    os.makedirs(path_temp_files)
else:
    for the_file in os.listdir(path_temp_files):
        file_path = os.path.join(path_temp_files, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

#import mapnik
sys.path.append(config.get("mapnik","pythonPath")) 
import mapnik
kartenscale=6

print mapnik.fontscollectionpath
custom_fonts_dir = 'C:/Windows/Fonts/'
mapnik.register_fonts(custom_fonts_dir)

breitePx=1242
hoehePx=1756

teil_lon=8
teil_lat=8

minLon=int(config.get("1","minLon"))
maxLon=int(config.get("1","maxLon"))
minLat=int(config.get("1","minLat"))
maxLat=int(config.get("1","maxLat"))

stylesheet = "tempdata/stylesheet.xml"

#search and replace the 
print('Starte tile setup')

m = mapnik.Map(breitePx,hoehePx)
mapnik.load_map(m, stylesheet)
print "BUFFER SIZE"
print m.buffer_size
m.background = mapnik.Color('white')
#definiere die Spaltenbreite
breiteLon=(maxLon-minLon)/teil_lon
breiteLat=(maxLat-minLat)/teil_lat
count=0

startLat=minLat
startLon=minLon

for i in range(teil_lat):
    for i in range(teil_lon):
        printBounds(startLat, startLon, (startLat+breiteLat), (startLon+breiteLon))
        renderBounds(startLat, startLon, (startLat+breiteLat), (startLon+breiteLon),path_temp_files+"/"+str(count))
        startLat+=breiteLat
        count+=1
    startLat=minLat
    startLon+=breiteLon
    
print "grosse Karte binished!"


killDataBaseServer()

Datenbank existiert
C:\xampp\htdocs\OSM-Standortkarten\mapnik-win-v2.2.0\lib\mapnik/fonts
Starte tile setup
BUFFER SIZE
878
828086,5925795,872419,5988473
This is the Bounding Box:  Box2d(828086.0,5925793.92432,872419.0,5988474.07568)  and the scale:  35.6948470209
872419,5925795,916752,5988473
This is the Bounding Box:  Box2d(872419.0,5925793.92432,916752.0,5988474.07568)  and the scale:  35.6948470209
916752,5925795,961085,5988473
This is the Bounding Box:  Box2d(916752.0,5925793.92432,961085.0,5988474.07568)  and the scale:  35.6948470209
961085,5925795,1005418,5988473
This is the Bounding Box:  Box2d(961085.0,5925793.92432,1005418.0,5988474.07568)  and the scale:  35.6948470209
1005418,5925795,1049751,5988473
This is the Bounding Box:  Box2d(1005418.0,5925793.92432,1049751.0,5988474.07568)  and the scale:  35.6948470209
1049751,5925795,1094084,5988473
This is the Bounding Box:  Box2d(1049751.0,5925793.92432,1094084.0,5988474.07568)  and the scale:  35.6948470209
1094084,5925795,1138

In [ ]:
r=requests.post(config.get("webservice","url"),
                auth=HTTPBasicAuth(config.get("webservice","username"),config.get("webservice","password")),
                data={"command":"fetchAllPrintingQueue"},
               )
if r.status_code == 200:
    queue=json.loads(r.text)
    for job in queue:
        print job
        #start db and wait some seconds
        
        #import mapnik
        sys.path.append(config.get("mapnik","pythonPath")) 
        import mapnik
        
else:
    print "konnte Printqueue nicht holen",r.status_code,r.text
